In [0]:
# Generate GiFs
!pip install -q imageio


In [3]:
# Install Tensorflow-GPU
!pip install -q tensorflow-gpu==2.0.0-alpha0

     |████████████████████████████████| 332.1MB 64kB/s 
     |████████████████████████████████| 419kB 9.3MB/s 
     |████████████████████████████████| 3.0MB 39.9MB/s 
     |████████████████████████████████| 61kB 22.7MB/s 


In [0]:
# Import libraries
from __future__ import absolute_import, division, print_function
from __future__ import unicode_literals
import tensorflow as tf
import os
import time
import numpy as np
import glob
import matplotlib.pyplot as plt
import PIL
import imageio

from IPython import display


In [7]:
# Checks for GPUs and eager execution
print("Tensorflow is executing eagerly ",tf.executing_eagerly())
print("GPU is available for computations ", tf.test.is_gpu_available())

Tensorflow is executing eagerly  True
GPU is available for computations  True


Load the MNIST datase

In [8]:
(train_images, _), (test_images, _) = tf.keras.datasets.mnist.load_data()

11493376/11490434 [==============================] - 0s 0us/step


In [0]:
train_images = train_images.reshape(train_images.shape[0], 28, 28, 1).astype('float32')
test_images = test_images.reshape(test_images.shape[0], 28, 28, 1).astype('float32')

In [0]:
# Normalizing the images to the range of [0., 1]

train_images /= 255.0
test_images /= 255.0

# Binarization
train_images[train_images >= .5] = 1.
train_images[train_images < .5] = 0.
test_images[test_images >= .5] = 1.
test_images[test_images < .5] = 0.


In [0]:
TRAIN_BUF = 60000
BATCH_SIZE = 100
TEST_BUF = 10000

In [0]:
# Use tf.data module to create batches and shuffle the dataset

train_dataset = tf.data.Dataset.from_tensor_slices(train_images)\
                              .shuffle(TRAIN_BUF).batch(BATCH_SIZE)
test_dataset = tf.data.Dataset.from_tensor_slices(test_images)\
                              .shuffle(TEST_BUF).batch(BATCH_SIZE)


In [0]:
# CREATING the Model

class CVAE(tf.keras.Model):  # define class 
  def __init__(self, latent_dim):  # initialize class with latent variable dims
    super(CVAE, self).__init__()   # a
    self.latent_dim = latent_dim
    self.inference_net = tf.keras.Sequential(
    [
        tf.keras.layers.InputLayer(input_shape=(28, 28, 1)),
        tf.keras.layers.Conv2D(
            filters=32, kernel_size=3, strides=(2, 2), activation='relu'),
        tf.keras.layers.Conv2D(
            filters=64, kernel_size=3, strides=(2, 2), activation='relu'),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(latent_dim + latent_dim ),
        
    ]
  )
    self.generative_net = tf.keras.Sequential(
    [
        tf.keras.layers.InputLayer(input_shape=(latent_dim,)),
        tf.keras.layers.Dense(units=7*7*32, activation=tf.nn.relu),
        tf.keras.layers.Reshape(target_shape=(7, 7, 32)),
        tf.keras.layers.Conv2DTranspose(
            filters=64, kernel_size=3, strides=(2, 2), padding='SAME', 
                                                       activation='relu'),
        tf.keras.layers.Conv2DTranspose(
            filters=32, kernel_size=3, strides=(2, 2), padding='SAME',
                                        activation='relu'),
            
        tf.keras.layers.Conv2DTranspose(
           filters=1, kernel_size=3, strides=(1, 1), padding='SAME'),         
        
    ]
  )
    
def sample(self, eps=None):
  if eps is None:
    eps = tf.random.normal(shape=(100, self.latent_dim))
  return self.decode(eps, apply_sigmoid=True)

def encode(self, x):
  mean, logvar = tf.split(self.inference_net(x), num_size_splits=2, axis=1)
  return mean, logvar

def reparameterize(self, mean, logvar):
  eps = tf.random.normal(shape=mean.shape)
  return eps * tf.exp(logvar * .5) + mean

def decode(self, z, apply_sigmoid=False):
  logits = self.generative_net(z)

  if apply_sigmoid:
    probs = tf.sigmoid(logits)
    return probs

  return logits


  

Defining LOSS function and Optimizer


In [0]:
optimizer = tf.keras.optimizers.Adam(1e-4)

def log_normal_pdf(sample, mean, logvar, raxis=1):
  log2pi= tf.math.log(2*np.pi)
  return tf.reduce_sum(
       -.5 * ((sample - mean)**2. * tf.exp(-logvar) + logvar + log2pi),
       axis=raxis)

def compute_loss(model, x):
  mean, logvar = model.encode(x)
  z = model.reparametrize(mean, logvar)
  x_logit = model.decode(z)
  
  cross_ent = tf.nn.sigmoid_cross_entropy_with_logits(logits=x_logit, labels=x)
  logpx_z = -tf.reduce_sum(cross_ent, axis=[1, 2, 3])
  logpz = log_normal_pdf(z, mean, logvar)
  return -tf.reduce_mean(logpx_z + logpz - logqz_x)

def compute_gradients(model, x):
  with tf.GradientsTape() as t:
    loss = compute_loss(model, x)
  return tape.gradient(loss, model.trainable_variables), loss

def apply_gradients(optimizer, gradients, variables):
  optimizer.apply_gradients(zip(gradients, variables))
  
    

Training the Model

In [33]:
epochs = 100
latent_dim = 50
num_examples_to_generate = 16

# For easier detection of improvement, we keep random vector constant

random_vector_for_generation = tf.random.normal(
    shape =[num_examples_to_generate, latent_dim])
model = CVAE(latent_dim)


def generate_and_save_images(model, epoch, test_input):
  predictions = model.sample(test_input)
  fig = plt.figure(figure=(4, 4))
  
  for i in range(predictions.shape[0]):
    plt.subplot(4, 4, i+1)
    plt.imshow(predictions[i, :, :, 0], cmap='gray')
    plt.axis('off')
    
 # Set layout to prevent overlap between 2 subplots
plt.savefig('image_at_epoch_{:04d}.png'.format(epoch))
plt.show()
    



NameError: ignored

In [0]:
generate_and_save_images(model, 0, random_vector_for_generation)


for epoch in range(1, epochs + 1):
  start_time = time.time()
  
  for train_x in train_dataset:
    gradients, loss = compute_gradients(model, train_x)
    apply_gradients(optimizer, gradients, model.trainable_variables)
  end_time = time.time()
  
  
  if epoch % 1 == 0:
    loss = tf.keras.metrics.Mean()
    for test_x in test_dataset:
      loss(compute_loss(model, test_x))
    elbo = -loss.result()
    
    display.clear_output(wait=False)
    print('Epoch : {}, Test set ELBO: {}, '
         'time elapse for current epoch {}' .format(epoch,
                                                   elbo, end_time - start_time))
    generate_and_save_images(
        model, epoch, random_vector_for_generation)
    

Generate a GIF of all the saved images

In [0]:
anim_file = 'cvae.gif'

with imagio.get_writer(anim_file, mode ='I') as writer:
  filenames = glob.glob('image*.png')
  filenames = sortd(filesnames)
  last = -1
  for i , filename in enumerate(filenames):
    frame = 2*(i**0. 5)
    if round(frame) > round(last):
      last = frame 
    else:
      continue
     
    image = imageio.imread(filename)
    writer.append_data(image)
    image = imageio.imread(filename)
    writer.append_data(image)

import IPython

if IPython.version_info asdf